In [1]:
from création_data import param_Dataset
import fonction_model as f
from IPython.display import display, Audio
from scipy.io.wavfile import write
import pickle


In [2]:
Fe = 2**15

## Medium 1:

In [3]:
Fichier = ""
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = True, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = False)

94634.57769887915 104587.731763282


In [4]:
T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[0]
Q,F,F_2 = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe,obs = True) 

[[ 9.86675222e-01  1.33247072e-02 -1.33248493e-02 ... -4.55668155e-02
   3.38118938e-02  2.04781508e-02]
 [ 1.33247072e-02  9.86646596e-01  1.32960109e-02 ...  4.55911668e-02
  -3.38299631e-02 -2.04890945e-02]
 [-1.33248493e-02  1.32960109e-02  9.86646313e-01 ... -4.55424644e-02
   3.37938246e-02  2.04672072e-02]
 ...
 [-5.35091784e-05  5.35377742e-05 -5.34805829e-05 ...  9.99816990e-01
   1.35798942e-04  8.22465382e-05]
 [ 3.97053565e-05 -3.97265754e-05  3.96841378e-05 ...  1.35798942e-04
   9.99899233e-01 -6.10293079e-05]
 [ 2.40475226e-05 -2.40603738e-05  2.40346715e-05 ...  8.22465382e-05
  -6.10293079e-05  9.99963038e-01]] [[-1.56473067e-05  1.56472232e-05 -1.56473901e-05 ... -5.35091784e-05
   3.97053565e-05  2.40475226e-05]
 [ 1.56472232e-05 -1.56809217e-05  1.56135251e-05 ...  5.35377742e-05
  -3.97265754e-05 -2.40603738e-05]
 [-1.56473901e-05  1.56135251e-05 -1.56812548e-05 ... -5.34805829e-05
   3.96841378e-05  2.40346715e-05]
 ...
 [-5.35091784e-05  5.35377742e-05 -5.3480582

In [ ]:
display(Audio(F_2,rate=Fe))

In [ ]:
from scipy.io import wavfile, loadmat
C1P1mat = loadmat("../../Mesures/Mesures_2023.01.27_LAM_normalise/Medium_1/Corde_Acier/C_2_P_1.mat")
acc = C1P1mat["acc_t"].reshape(-1)
fs2 = int(C1P1mat["fs"].reshape(-1))

In [ ]:
display(Audio(acc,rate = fs2))

In [ ]:
import numpy as np
Nfft = len(acc)
Y2 = np.fft.rfft(acc, Nfft)
fy2 = np.fft.rfftfreq(Nfft,1/fs2)

In [ ]:

Q_f = np.fft.rfft(Q)
F_f = np.fft.rfft(F)
F_f2 = np.fft.rfft(F_2)

freq = np.fft.rfftfreq(len(F)) * Fe

import matplotlib.pyplot as plt
%matplotlib widget

plt.figure()
plt.semilogx(freq,20*np.log10(np.abs(Q_f)),label='accel')
plt.semilogx(freq,20*np.log10(np.abs(F_f)),label='Force_corde')
plt.semilogx(freq,20*np.log10(np.abs(F_f2)),label='Force_table')
plt.semilogx(freq,20*np.log10(np.abs(F_f2 / F_f)),label='Force dif')

plt.semilogx(fy2, 20*np.log10(np.abs(Y2)),label='Mesure')

plt.xlim(20,12000)
plt.legend()
plt.grid()
plt.show()

In [ ]:
display(Audio(Q,rate=Fe))

In [ ]:
for i in range(1):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe) 
    display(Audio(F,rate=Fe))  
    #write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly 

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## medium 2

In [ ]:
Fichier = "1ier_data_set/medium_2/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = True, 
                                             metal_T = False,
                                             plexi_T = False)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## Acier : 

In [ ]:
Fichier = "1ier_data_set/acier/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = True,
                                             plexi_T = False)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)  
    #display(Audio(F,rate=Fe)) 
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)


## Plexi : 

In [ ]:
Fichier = "1ier_data_set/plexi/"
N_sample, param_dataset, xinB = param_Dataset(N_sample=200,
                                             article_C = True,
                                             acier_1C = False,
                                             acier_2C = False, 
                                             medium_1T = False, 
                                             medium_2T = False, 
                                             metal_T = False,
                                             plexi_T = True)

In [ ]:
for i in range(N_sample):
    T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly = param_dataset[i]
    F = f.Main(T,rho_l,L,E_corde,I,h,E_nu,rhoT,Lx,Ly,xinB,Fe)   
    write(Fichier + str(i) + '.wav',rate=Fe ,data = F / max(F))

In [ ]:
dico = {}
for i in range(N_sample):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = param_dataset[i][0]
    dico[kle]['rho_l_corde'] = param_dataset[i][1]
    dico[kle]['L_corde'] = param_dataset[i][2]
    dico[kle]['E_corde'] = param_dataset[i][3]
    dico[kle]['I_corde'] = param_dataset[i][4]
    dico[kle]['h_table'] = param_dataset[i][5]
    dico[kle]['E_nu_table'] = param_dataset[i][6]
    dico[kle]['rho_table'] = param_dataset[i][7]
    dico[kle]['Lx_table'] = param_dataset[i][8]
    dico[kle]['Ly_table'] = param_dataset[i][9]

with open(Fichier + 'parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)
